In [0]:
import json 
import requests
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
usda_api_key = "41C0481B-3390-3B15-8AFA-C1E982024C76"
base_url = "https://quickstats.nass.usda.gov/api/api_GET/"


In [0]:
dbutils.widgets.text("commodity","corn")
commodity = dbutils.widgets.get("commodity").upper()

In [0]:
dbutils.widgets.text("statistic","YIELD")
statistic = dbutils.widgets.get("statistic").upper()

In [0]:
params = {
        "key": usda_api_key,
        "format": "json",
        "commodity_desc": f"{commodity}",
        "statisticcat_desc": f"{statistic}",
        "freq_desc": "ANNUAL",
        "agg_level_desc": "STATE",
        "year__GE":"2019"
    }
response = requests.get(base_url,params=params)
data_usda = response.json()
print(data_usda)

In [0]:
%sql
CREATE VOLUME if not exists priceriskanalysis.bronze.usda_bronze;

In [0]:
dbutils.widgets.text("catalog_name","priceriskanalysis")
catalog_name = dbutils.widgets.get("catalog_name")

In [0]:
if statistic == "YIELD":
    bronze_path = f"/Volumes/{catalog_name}/bronze/usda_bronze/yeild/"
elif statistic == "PRODUCTION":
    bronze_path = f"/Volumes/{catalog_name}/bronze/usda_bronze/production/"
elif statistic == "AREA HARVESTED":
    bronze_path = f"/Volumes/{catalog_name}/bronze/usda_bronze/area_harvested/"
elif statistic == "PRICE RECEIVED":
    bronze_path = f"/Volumes/{catalog_name}/bronze/usda_bronze/price_received/"
else:
    raise ValueError("Unknown Statistic")


In [0]:
from datetime import datetime
current_time_stamp = datetime.now()
file_name = f"usda_{commodity}_{statistic}_{current_time_stamp}.json"
with open(bronze_path + file_name,'w') as file:
    json.dump(data_usda,file)